In [7]:
spark

StatementMeta(testspark4298, 0, 8, Finished, Available)

In [8]:
#Access key
spark.conf.set("fs.azure.account.auth.type.adlsstorage4298.dfs.core.windows.net","SharedKey")

StatementMeta(testspark4298, 0, 9, Finished, Available)

In [9]:
spark.conf.set("fs.azure.account.key.adlsstorage4298.dfs.core.windows.net","aWexrLVdOokH3MYN416YGi/feNTsJHD8GQjHipz74u4BVJMw108gRt+j1/wzD7LMOpP1z8/uGi36+AStkSWF0w==")


StatementMeta(testspark4298, 0, 10, Finished, Available)

In [33]:
from pyspark.sql.types import *
schemaTransData = StructType([
    StructField("txnid", IntegerType(), True),
    StructField("txndate", StringType(), True),
    StructField("custid", IntegerType(), True),
    StructField("amount", DoubleType(), True),
    StructField("category", StringType(), True),
    StructField("subcategory", StringType(), True),
    StructField("city", StringType(), True),
    StructField("state", StringType(), True),
    StructField("txntype", StringType(), True),
                    ])
dataFrame1 = spark.read.option("header",False).schema(schemaTransData).option("delimiter",",").csv("abfss://adlsdata@adlsstorage4298.dfs.core.windows.net/transactions/txns")
dataFrame1.show(5)

StatementMeta(testspark4298, 0, 34, Finished, Available)

+-----+----------+-------+------+------------------+--------------------+-----------+----------+-------+
|txnid|   txndate| custid|amount|          category|         subcategory|       city|     state|txntype|
+-----+----------+-------+------+------------------+--------------------+-----------+----------+-------+
|    0|06-26-2011|4007024| 40.33|Exercise & Fitness|Cardio Machine Ac...|Clarksville| Tennessee| credit|
|    1|05-26-2011|4006742|198.44|Exercise & Fitness|Weightlifting Gloves| Long Beach|California| credit|
|    2|06-01-2011|4009775|  5.58|Exercise & Fitness|Weightlifting Mac...|    Anaheim|California| credit|
|    3|06-05-2011|4002199|198.19|        Gymnastics|    Gymnastics Rings|  Milwaukee| Wisconsin| credit|
|    4|12-17-2011|4002613| 98.81|       Team Sports|        Field Hockey|Nashville  | Tennessee| credit|
+-----+----------+-------+------+------------------+--------------------+-----------+----------+-------+
only showing top 5 rows



In [34]:
dataFrame1.printSchema()

StatementMeta(testspark4298, 0, 35, Finished, Available)

root
 |-- txnid: integer (nullable = true)
 |-- txndate: string (nullable = true)
 |-- custid: integer (nullable = true)
 |-- amount: double (nullable = true)
 |-- category: string (nullable = true)
 |-- subcategory: string (nullable = true)
 |-- city: string (nullable = true)
 |-- state: string (nullable = true)
 |-- txntype: string (nullable = true)



In [36]:
schemaCustdata= StructType([
    StructField("custid", IntegerType(), True),
    StructField("fname", StringType(), True),
    StructField("lname", StringType(), True),
    StructField("age", IntegerType(), True),
    StructField("profession", StringType(), True)
                    ])
dataFrame2 = spark.read.option("header",False).schema(schemaCustdata).option("delimiter",",").csv("abfss://adlsdata@adlsstorage4298.dfs.core.windows.net/custdata/custs")
dataFrame2.show(5)

StatementMeta(testspark4298, 0, 37, Finished, Available)

+-------+--------+-------+---+--------------------+
| custid|   fname|  lname|age|          profession|
+-------+--------+-------+---+--------------------+
|4000001|Kristina|  Chung| 55|               Pilot|
|4000002|   Paige|   Chen| 74|             Teacher|
|4000003|  Sherri| Melton| 34|         Firefighter|
|4000004|Gretchen|   Hill| 66|Computer hardware...|
|4000005|   Karen|Puckett| 74|              Lawyer|
+-------+--------+-------+---+--------------------+
only showing top 5 rows



In [37]:
dataFrame2.printSchema()

StatementMeta(testspark4298, 0, 38, Finished, Available)

root
 |-- custid: integer (nullable = true)
 |-- fname: string (nullable = true)
 |-- lname: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- profession: string (nullable = true)



In [38]:
dataFrame1.registerTempTable('Transactions')
dataFrame2.registerTempTable('Customers')
spark.sql("show tables").show()

StatementMeta(testspark4298, 0, 39, Finished, Available)

+--------+------------+-----------+
|database|   tableName|isTemporary|
+--------+------------+-----------+
|        |   customers|       true|
|        |transactions|       true|
+--------+------------+-----------+



In [40]:
#a. Find the total revenue generated based on category
spark.sql("select category,sum(amount) as total_revenue from transactions group by category order by total_revenue").show()

StatementMeta(testspark4298, 0, 41, Finished, Available)

+--------------------+--------------------+
|            category|       total_revenue|
+--------------------+--------------------+
|             Dancing|   820886.2499999998|
|             Puzzles|  1227776.4400000002|
|          Air Sports|   2080245.090000001|
|       Combat Sports|   3305832.759999997|
|      Racquet Sports|   3314022.080000001|
|             Jumping|  4107379.3399999966|
|Outdoor Play Equi...|   5765130.969999998|
|        Indoor Games|   5774627.900000003|
|          Gymnastics|   6617582.299999998|
|       Winter Sports|   6628206.310000003|
|               Games|   7417431.410000009|
|        Water Sports| 1.074843876000002E7|
|         Team Sports|1.2405851640000027E7|
|  Exercise & Fitness|1.4890130299999973E7|
|  Outdoor Recreation| 1.736480985999999E7|
+--------------------+--------------------+



In [41]:
#b. Find the total number of transactions done by card
spark.sql("select count(txnid) as total_card_transactions from transactions where txntype='credit'").show()

StatementMeta(testspark4298, 0, 42, Finished, Available)

+-----------------------+
|total_card_transactions|
+-----------------------+
|                 863585|
+-----------------------+



In [42]:
#c. Find the highest selling category
spark.sql("select category,sum(amount) as total_revenue from transactions group by category order by total_revenue desc limit 1").show()

StatementMeta(testspark4298, 0, 43, Finished, Available)

+------------------+--------------------+
|          category|       total_revenue|
+------------------+--------------------+
|Outdoor Recreation|1.7364809859999992E7|
+------------------+--------------------+



In [43]:
#d. Find the lowest selling subcategory in highest selling category
spark.sql("select subcategory,sum(amount) as total_revenue from transactions where category='Outdoor Recreation' group by subcategory order by total_revenue asc limit 1").show()

StatementMeta(testspark4298, 0, 44, Finished, Available)

+-----------+-----------------+
|subcategory|    total_revenue|
+-----------+-----------------+
| Equestrian|806128.8099999996|
+-----------+-----------------+



In [45]:
#e. Find the total number of transactions done by customers with profession = Carpenter
spark.sql("select count(txnid) as total_transactions from transactions join customers on transactions.custid=customers.custid where customers.profession='Carpenter'").show()

StatementMeta(testspark4298, 0, 46, Finished, Available)

+------------------+
|total_transactions|
+------------------+
|             18231|
+------------------+

